In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling

In [ ]:
%matplotlib inline

In [ ]:
!dir

In [ ]:
%cd .\decision_tree

In [ ]:
!dir

In [ ]:
df = pd.read_csv(".\\credit_cards_dataset.csv")

In [ ]:
df['EDUCATION'].plot.hist()

In [ ]:
#df.describe()

In [ ]:
#df['PAY_AMT4']

In [ ]:
#df.tail(5)

In [ ]:
### EDA & Featuring

In [ ]:
#df.describe()

In [ ]:
'''
df = pd.DataFrame(
    np.random.rand(100, 5),
    columns=['a', 'b', 'c', 'd', 'e']
)
'''

In [ ]:
#df.profile_report(style={'full_width':True})
df.profile_report()

In [ ]:
#print("Original shape of the data: "+ str(df.shape))
features_names = df.columns

In [ ]:
#df.describe()

In [ ]:
X = df.drop('default.payment.next.month', axis =1).values
y = df['default.payment.next.month'].values
print(X.shape)
print(y.shape)

Split my data into training and testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42, shuffle=True)

Instantiate the random forest model with 200 trees

In [ ]:
rf = RandomForestClassifier(n_estimators=200, criterion='entropy', max_features='log2', max_depth=15)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_predict = rf.predict(X_test)

In [ ]:
#y_predict.reshape(-1,1).shape

Check feature importance


In [ ]:
sorted(zip(rf.feature_importances_, features_names), reverse=True)

In [ ]:
## plot the importances ##
import matplotlib.pyplot as plt
importances = rf.feature_importances_


indices = np.argsort(importances)[::-1]
plt.figure(figsize=(12,6))
plt.title("Feature importances by DecisionTreeClassifier")
plt.bar(range(len(indices)), importances[indices], color='lightblue',  align="center")
plt.step(range(len(indices)), np.cumsum(importances[indices]), where='mid', label='Cumulative')
plt.xticks(range(len(indices)), features_names[indices], rotation='vertical',fontsize=14)
plt.xlim([-1, len(indices)])
#plt.show()

# Making my prediction and seeing how well my model predicted by checking recall, precision, F1 score and making a confusion matrix.

Recall -tells us generally or overall how well our model predicted based on 
        the total of how much it correctly predicted /correctly predicted +                           how many where actually right but predicted wrong.
                        
                        formula = TP/TP+FN
                        
Precision -tells us or gives us true measure how well our model predicted it                              shows correctly predicted /correctly predicted + how many the model                            predicted to be positive but where false.

                       formula = TP/TP+FP
                       
F1 score - gives us a mean of precision and recall, a sumarization of how well it                        did in respect to recall and precision.


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
X_test.shape

In [ ]:
#Make my predictions
y_prediction = rf.predict(X_test)

In [ ]:
y_probability = rf.predict_proba(X_test)

In [ ]:
y_probability.shape

In [ ]:
### Grid search
### https://towardsdatascience.com/algorithms-for-hyperparameter-optimisation-in-python-edda4bdb167

In [ ]:
'''
rf_params = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth':np.arange(5,15,1)
}

gs_random = GridSearchCV(estimator=rf, param_grid=rf_params, cv= 5)
gs_random.fit(X_test, y_prediction)
print(gs_random.best_params_)
'''

In [ ]:
print("Recall score:"+ str(recall_score(y_test, y_prediction)))

In [ ]:
y_prediction.reshape(-1,1)

In [ ]:
# This shows overall acuracy of how well it will predict defualt or non_default
# The scores corresponding to every class will tell you the accuracy of the classifier 
# in classifying the data points in that particular class compared to all other classes.
# The support is the number of samples of the true response that lie in that class.

print(classification_report(y_test, y_prediction,
                            target_names=["non_default", "default"]))

In [ ]:
# Creating confusion matrix would give us a ration of non-default compared 
# to default.

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import numpy as np

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_prediction)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Non_Default','Default'], normalize=False,
                      title='Non Normalized confusion matrix')

# Explanation of this confusion matrix
In our confusion matrix, the non-default classification has a total of 2,158 points and defualt clssification has a total of 7,742 points.

It correctly identified 7239 points as default and 503 points as non-default.

Non_default classification incorectly predicted 1480 points as default  and correctly classified 678 points as Non defualt. 


In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Defualt', 'Non_default'], normalize=True,
                      title='Normalized confusion matrix')

In [ ]:
import pickle
filename = 'RandomForest_model.sav'
pickle.dump(rf, open(filename, 'wb'))